LlamaIndex は、データに対して単純なものから高度なものまでのワークフローを調整するために、さまざまなモジュールを連結できる宣言型クエリ API を提供します。

これは、QueryPipeline 抽象化を中心にしています。 さまざまなモジュール (LLM からプロンプト、レトリーバー、その他のパイプラインまで) をロードし、それらをすべてまとめてシーケンシャル チェーンまたは DAG に接続し、end2end で実行します。

注: これらすべてのワークフローは、宣言的なパイプライン抽象化を行わずに (モジュールを命令的に使用し、独自の関数を作成することで) オーケストレーションできます。 では、QueryPipeline の利点は何でしょうか?

- より少ないコード行/ボイラープレートで一般的なワークフローを表現します
- 可読性の向上
- 一般的なローコード/ノーコード ソリューション (LangFlow など) とのより優れた同等性/より優れた統合ポイント
- [将来] 宣言型インターフェイスにより、パイプライン コンポーネントのシリアル化が容易になり、パイプラインの移植性が提供され、さまざまなシステムへの展開が容易になります。

このクックブックでは、QueryPipeline インターフェイスの概要を説明し、取り組むことができる基本的なワークフローをいくつか示します。

- プロンプトと LLM を連結する
- クエリの書き換え (プロンプト + LLM) と取得を連結する
- 完全な RAG クエリ パイプラインを連結します (クエリの書き換え、取得、再ランキング、応答の合成)。
- カスタムクエリコンポーネントの設定

Here we setup some data + indexes (from PG's essay) that we'll be using in the rest of the cookbook.

`llama-index-callbacks-arize-phoenix>0.1.3`には`Python >=3.8.1,<3.12`というバージョン縛りがある。
このNotebookでは Python==3.12.2 を利用していて、依存解決できないため、arize-phoenixの利用は断念。

In [1]:
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate
from llama_index.core import (
    VectorStoreIndex, ServiceContext, SimpleDirectoryReader, load_index_from_storage
)

In [3]:
reader = SimpleDirectoryReader("../data/paul_graham")

In [8]:
docs = reader.load_data()
print(docs[0].get_content())



What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in t

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

from llama_index.core.storage import StorageContext

if not os.path.exists("storage"):
    index = VectorStoreIndex.from_documents(docs)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    # load index
    index = load_index_from_storage(storage_context=storage_context, index_id="vector_index")

このセクションでは、LLM を使用してプロンプトを連鎖させる非常に単純なワークフローを示します。

初期化時にチェーンを定義するだけです。 これは、コンポーネントが純粋にシーケンシャルであるクエリ パイプラインの特殊なケースであり、出力を次の入力に適した形式に自動的に変換します。

In [15]:
# chaining basic prompts
# prompt_str = "Please generate related movies to {movie_name}"
# QueryPipelineを使用
prompt_str = "{movie_name} に関連する映画を生成してください"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model=os.getenv("CHAT_MODEL_GPT4"))

p = QueryPipeline(
    chain=[prompt_tmpl, llm],
    verbose=True
)

In [16]:
output = p.run(movie_name="Pulp Fiction")

> Running module 097c091e-03aa-4161-97cd-1f1a9ae812e4 with input: 
movie_name: Pulp Fiction

> Running module e57417a4-afd4-483a-b448-8254c626c7d1 with input: 
messages: Pulp Fiction に関連する映画を生成してください



In [17]:
print(str(output))

assistant: 『Pulp Fiction』は、クエンティン・タランティーノ監督による1994年のカルトクラシック映画で、非線形の物語構造、鮮やかなダイアログ、そして記憶に残るキャラクターで知られています。この映画に影響を受けたり、類似したテーマやスタイルを持つ映画をいくつか挙げます：

1. **レザボア・ドッグス (Reservoir Dogs)** - 1992年
   - タランティーノのデビュー作であり、『Pulp Fiction』と同様に非線形の物語と独特の対話が特徴です。

2. **ジャッキー・ブラウン (Jackie Brown)** - 1997年
   - タランティーノ監督作品で、彼の映画の中でも比較的落ち着いたトーンを持ちつつ、強烈なキャラクターと巧妙なプロットが楽しめます。

3. **シン・シティ (Sin City)** - 2005年
   - フランク・ミラーの同名のグラフィックノベルを基にしたこの映画は、『Pulp Fiction』と同じく複数の物語が交錯するスタイルを採用しています。

4. **スナッチ (Snatch)** - 2000年
   - ガイ・リッチー監督によるこの映画は、複数の登場人物と入り組んだプロット、速いテンポのエディティングが特徴で、『Pulp Fiction』のファンには魅力的かもしれません。

5. **トゥルー・ロマンス (True Romance)** - 1993年
   - タランティーノが脚本を書いたこの映画は、彼の特徴的なダイアログと暴力的なアクションが含まれており、『Pulp Fiction』の前触れとも言える作品です。

これらの映画は、『Pulp Fiction』が持つ独特のスタイルやテーマを異なる形で表現しており、同じような映画体験を求める視聴者におすすめです。


# if you implemented this imperatively - you can still do so
# just make sure you format the prompt and call the right method on the LLM
# QueryPipelineを使用しない方法。コードでフローを制御する
from llama_index.core.llms import ChatMessage, MessageRole

# try chaining basic prompts
prompt_str = "Please generate related movies to {movie_name}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model=os.getenv("CHAT_MODEL_GPT4"))

# format prompt, pass to LLM
movie_name = "Pulp Fiction"
full_prompt_tmpl = prompt_tmpl.format(movie_name=movie_name)
response = llm.chat([ChatMessage(content=full_prompt_tmpl, role=MessageRole.USER)])

print(str(response))

Let's parse the outputs into a structured Pydantic object.

In [20]:
from typing import List
from pydantic import BaseModel, Field
from llama_index.core.output_parsers import PydanticOutputParser

class Movie(BaseModel):
    """Object representing a single movie."""
    
    name: str = Field(..., description="Name of the movie.")
    year: int = Field(..., description="Year of the movie.")
    
class Movies(BaseModel):
    """Object representing a list of movies."""
    
    movies: List[Movie] = Field(..., description="List of movies.")

llm = OpenAI(model=os.getenv("CHAT_MODEL_GPT4"))
# PydanticOutputParser：JSONオブジェクトをPydanticオブジェクトに変換する
output_parser = PydanticOutputParser(Movies)
json_prompt_str = """\
Please generate related movies to {movie_name}.
"""
json_prompt_str = output_parser.format(json_prompt_str)

In [21]:
print(json_prompt_str)

Please generate related movies to {movie_name}.



Here's a JSON schema to follow:
{{"$defs": {{"Movie": {{"description": "Object representing a single movie.", "properties": {{"name": {{"description": "Name of the movie.", "title": "Name", "type": "string"}}, "year": {{"description": "Year of the movie.", "title": "Year", "type": "integer"}}}}, "required": ["name", "year"], "title": "Movie", "type": "object"}}}}, "description": "Object representing a list of movies.", "properties": {{"movies": {{"description": "List of movies.", "items": {{"$ref": "#/$defs/Movie"}}, "title": "Movies", "type": "array"}}}}, "required": ["movies"], "title": "Movies", "type": "object"}}

Output a valid JSON object but do not repeat the schema.


In [22]:
# add JSON spec to prompt template
json_prompt_tmpl = PromptTemplate(json_prompt_str)

p = QueryPipeline(
    chain=[json_prompt_tmpl, llm, output_parser],
    verbose=True
)
output = p.run(movie_name="Toy Story")

> Running module c2861a61-e74a-4474-bb89-7c879e981f6b with input: 
movie_name: Toy Story

> Running module 7f2252b9-53a4-430c-a012-88ec3cb4868f with input: 
messages: Please generate related movies to Toy Story.



Here's a JSON schema to follow:
{"$defs": {"Movie": {"description": "Object representing a single movie.", "properties": {"name": {"description": "Name ...

> Running module 535ff1cc-bd12-4037-95ce-12b3cb46f459 with input: 
input: assistant: {
  "movies": [
    {
      "name": "Toy Story 2",
      "year": 1999
    },
    {
      "name": "Toy Story 3",
      "year": 2010
    },
    {
      "name": "Toy Story 4",
      "year": 20...



In [23]:
output

Movies(movies=[Movie(name='Toy Story 2', year=1999), Movie(name='Toy Story 3', year=2010), Movie(name='Toy Story 4', year=2019), Movie(name='Monsters, Inc.', year=2001), Movie(name='Finding Nemo', year=2003), Movie(name='The Incredibles', year=2004), Movie(name='Cars', year=2006), Movie(name="A Bug's Life", year=1998), Movie(name='Up', year=2009), Movie(name='Inside Out', year=2015)])

クエリ パイプラインには LLM ストリーミング サポートがあります (単に as_query_component(streaming=True) を実行します)。 中間出力は自動変換され、最終出力はストリーミング出力になります。 以下にいくつかの例を示します。

In [28]:
prompt_str = "Please generate related movies to {movie_name}."
prompt_tmpl = PromptTemplate(prompt_str)

# add some subsequent prompts
prompt_str2 = """\
Here's some text:

{text}

Can you rewrite this with a summary of each movie?
"""

prompt_tmpl2 = PromptTemplate(prompt_str2)

llm = OpenAI(model=os.getenv("CHAT_MODEL_GPT4"))
llm_c = llm.as_query_component(streaming=True)

p = QueryPipeline(
    chain=[prompt_tmpl, llm_c, prompt_tmpl2, llm_c],
    verbose=True
)
# p = QueryPipeline(
#     chain=[prompt_tmpl, llm_c],
#     verbose=True
# )

In [29]:
output = p.run(movie_name="仁義なき戦い")
for o in output:
    print(o.delta, end="")

> Running module 9edb7f8f-b48f-4489-9282-9701f06c1bbb with input: 
movie_name: 仁義なき戦い

> Running module 09ae262c-340d-4d4f-a213-1148e2fdb94b with input: 
messages: Please generate related movies to 仁義なき戦い.

> Running module 4031f862-de2b-44cf-8d65-10f0567f6628 with input: 
text: <generator object llm_chat_callback.<locals>.wrap.<locals>.wrapped_llm_chat.<locals>.wrapped_gen at 0x16b216f80>

> Running module a2434a0f-2f6a-419f-8801-40b2261dbbeb with input: 
messages: Here's some text:

"仁義なき戦い" (Battles Without Honor and Humanity) is a classic yakuza film directed by Kinji Fukasaku, released in 1973. It's known for its gritty portrayal of the post-war yakuza under...

Certainly! Here's a rewritten version of the text with a brief summary for each movie:

"仁義なき戦い" (Battles Without Honor and Humanity) is a seminal yakuza film directed by Kinji Fukasaku, released in 1973. Renowned for its unflinching depiction of the post-war yakuza underworld in Japan, the film has spawned several sequels

In [30]:
# Feed streaming output to output parser
p = QueryPipeline(
    chain=[
        json_prompt_tmpl,
        llm.as_query_component(streaming=True),
        output_parser
    ],
    verbose=True
)
output = p.run(movie_name="トイ・ストーリー")
print(output)

> Running module 98b9f8cf-1ea5-4cfd-b65a-932ac3b14427 with input: 
movie_name: トイ・ストーリー

> Running module bd152c34-43e3-47bc-be6a-0775bdb0864a with input: 
messages: Please generate related movies to トイ・ストーリー.



Here's a JSON schema to follow:
{"$defs": {"Movie": {"description": "Object representing a single movie.", "properties": {"name": {"description": "Name o...

> Running module 72e8931d-a565-4227-ae9b-4a524f9ae33d with input: 
input: <generator object llm_chat_callback.<locals>.wrap.<locals>.wrapped_llm_chat.<locals>.wrapped_gen at 0x16b2169e0>

movies=[Movie(name='トイ・ストーリー2', year=1999), Movie(name='トイ・ストーリー3', year=2010), Movie(name='トイ・ストーリー4', year=2019), Movie(name='モンスターズ・インク', year=2001), Movie(name='ファインディング・ニモ', year=2003)]


ここでは、取得を開始する前に 2 つのプロンプトを通じて入力を送信する、もう少し複雑なワークフローを試します。

1. 指定されたトピックに関する質問を生成します。
2. 検索を改善するために、質問に幻覚を与えて答えます。

各プロンプトは 1 つの入力のみを受け取るため、QueryPipeline は LLM 出力をプロンプトに自動的にチェーンし、次に LLM にチェーンすることに注意してください。

In [33]:
# generate question regarding topic
prompt_str1 = "Please generate a concise question about Paul Graham's life regarding the following topic: {topic}"
prompt_tmpl1 = PromptTemplate(prompt_str1)
# use HyDE to hallucinate answer.
prompt_str2 = (
    "Please write a passage to answer the question.\n"
    "Try to include as many keywords as possible.\n"
    "\n"
    "\n"
    "{query_str}\n"
    "\n"
    "\n"
    'Passage:"""\n'
)
prompt_tmpl2 = PromptTemplate(prompt_str2)

llm = OpenAI(model=os.getenv("CHAT_MODEL_GPT4"))
retriever = index.as_retriever(similarity_top_k=5)
p = QueryPipeline(
    chain=[prompt_tmpl1, llm, prompt_tmpl2, llm, retriever],
    verbose=True
)

In [34]:
nodes = p.run(topic="college")
len(nodes)

> Running module 94c69262-ba8a-41a2-a608-93c6731b7df1 with input: 
topic: college

> Running module 111a1b91-48e7-499c-b061-59abc58a544b with input: 
messages: Please generate a concise question about Paul Graham's life regarding the following topic: college

> Running module b38a7f7a-2716-4a1d-9214-95d5c8c5e847 with input: 
query_str: assistant: What role did Paul Graham's education at Cornell University and Harvard University play in shaping his career in technology and entrepreneurship?

> Running module 57c64c0b-a9ca-4ddf-a099-072602eeaa7f with input: 
messages: Please write a passage to answer the question
Try to include as many keywords as possible.


What role did Paul Graham's education at Cornell University and Harvard University play in shaping his care...

> Running module c8d4845e-fba2-4d0a-ab63-f074de575f06 with input: 
input: assistant: Paul Graham's educational journey through Cornell University and Harvard University played a pivotal role in shaping his illustrious care

5